<span style="color:blue"> Бизнес-задача: 
- Предложить стратегию инвестиций в разработку игр, выявив связь между характеристиками игр (жанр, средний рейтинг, платформа) и их коммерческим успехом (продажи, выручка). </span>

In [67]:
import pandas as pd

In [68]:
games_rate_data = pd.read_csv('games.csv')
games_revenue_data = pd.read_csv('gamalytic_games.csv')

games_rate_data['esrb_rating'] = games_rate_data['esrb_rating'].fillna('No age rating')
games_rate_data = games_rate_data.rename(columns={'esrb_rating': 'age_rating'})

games_revenue = games_revenue_data.drop(columns=['steamId', 'unreleased', 'earlyAccess', 'publisherClass', 'publishers', 'headerUrl', 'steamUrl'])

`Заметим, что есть строки в двух датасетах, где игры одинаковые, но названия имею различия, поэтому необходимо грамотно обработать такие случаи, чтобы впоследствии соединить датасеты`

In [69]:
import re

def clean(column):
    column = str(column).lower()
    column = re.sub(r'[^a-z0-9]', ' ', column)
    column = re.sub(r'\s+', ' ', column).strip()
    return column

def numeric_part(n):
    return ' '.join(re.findall(r'\d+', str(n)))

gd = games_rate_data.copy()
gr = games_revenue.copy()

gd['filter_name'] = games_rate_data['name'].apply(clean)
gr['filter_name'] = games_revenue['name'].apply(clean)

gd = gd[gd['filter_name'] != '']
gr = gr[gr['filter_name'] != '']

gd['num_value'] = gd['filter_name'].apply(numeric_part)
gr['num_value'] = gr['filter_name'].apply(numeric_part)

joined = pd.merge(gr, gd, on=['filter_name', 'num_value'], how='inner', suffixes=('_rev', '_data'))

joined


,name_rev,firstReleaseDate,copiesSold,price,revenue,reviewScore,developers,filter_name,num_value,name_data,released,rating,age_rating,genres,platforms
0,Counter-Strike 2,2012-05-01T00:00:00.000Z,324828766,0.00,9898329661,86,Valve,counter strike 2,2,Counter-Strike 2,2023-09-27,3.58,Mature,Shooter,"Linux, PC"
1,Dota 2,2011-08-01T00:00:00.000Z,213939465,0.00,2091450944,81,Valve,dota 2,2,Dota 2,2013-07-09,3.07,No age rating,"Action, Massively Multiplayer","Linux, macOS, PC"
2,Apex Legends™,2020-11-04T05:00:00.000Z,72503084,0.00,1491767840,67,Respawn,apex legends,,Apex Legends,2019-02-04,3.64,Teen,"Action, Shooter","PlayStation 4, Nintendo Switch, macOS, PC, Xbo..."
3,Destiny 2,2019-10-01T04:00:00.000Z,38861886,0.00,901439163,78,Bungie,destiny 2,2,Destiny 2,2017-09-06,3.52,Teen,"Action, Shooter","PlayStation 4, Xbox One, PC, Web, Xbox Series ..."
4,Warframe,2013-03-25T04:00:00.000Z,47103348,0.00,879275446,87,Digital Extremes,warframe,,Warframe,2013-03-25,3.42,No age rating,"Action, Shooter, RPG, Massively Multiplayer","iOS, Xbox Series S/X, PlayStation 5, PlayStati..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5711,Mirror Maker,2019-01-17T05:00:00.000Z,83137,1.99,65436,71,KAGAMI WORKs,mirror maker,,Mirror Maker,2019-01-24,1.75,No age rating,"RPG, Simulation, Indie",PC
5712,Kromaia,2014-10-23T04:00:00.000Z,10378,9.99,65277,75,Kraken Empire,kromaia,,Kromaia,2014-10-22,0.00,No age rating,"Action, Shooter, Arcade, Indie","PlayStation 4, PC"
5713,DUCATI - 90th Anniversary,2016-06-09T04:00:00.000Z,11258,9.99,65188,74,Milestone S.r.l.,ducati 90th anniversary,90,DUCATI - 90th Anniversary,2016-06-09,2.60,No age rating,"Simulation, Sports, Racing","PlayStation 4, PC, Xbox One"
5714,The Chaos Engine,2013-08-29T04:00:00.000Z,26117,9.99,65092,68,"Abstraction Games,The Bitmap Brothers",the chaos engine,,The Chaos Engine,1993-03-01,3.67,No age rating,"Action, Shooter","Atari ST, Classic Macintosh, SNES, Commodore /..."


In [70]:
joined = joined.drop(columns=['num_value', 'name_rev', 'filter_name']) # Удаляем лишние (дублирующие) столбцы для дальнейшей работы с данными
cols = list(joined.columns)

cols


['firstReleaseDate',
 'copiesSold',
 'price',
 'revenue',
 'reviewScore',
 'developers',
 'name_data',
 'released',
 'rating',
 'age_rating',
 'genres',
 'platforms']

In [71]:
cols[0], cols[6] = cols[6], cols[0] # Восстанавливаем исходный порядок колонок
joined = joined[cols]

joined = joined.rename(columns={'name_data': 'name'}) # Возвращаем исходное название колонки

joined

,name,copiesSold,price,revenue,reviewScore,developers,firstReleaseDate,released,rating,age_rating,genres,platforms
0,Counter-Strike 2,324828766,0.00,9898329661,86,Valve,2012-05-01T00:00:00.000Z,2023-09-27,3.58,Mature,Shooter,"Linux, PC"
1,Dota 2,213939465,0.00,2091450944,81,Valve,2011-08-01T00:00:00.000Z,2013-07-09,3.07,No age rating,"Action, Massively Multiplayer","Linux, macOS, PC"
2,Apex Legends,72503084,0.00,1491767840,67,Respawn,2020-11-04T05:00:00.000Z,2019-02-04,3.64,Teen,"Action, Shooter","PlayStation 4, Nintendo Switch, macOS, PC, Xbo..."
3,Destiny 2,38861886,0.00,901439163,78,Bungie,2019-10-01T04:00:00.000Z,2017-09-06,3.52,Teen,"Action, Shooter","PlayStation 4, Xbox One, PC, Web, Xbox Series ..."
4,Warframe,47103348,0.00,879275446,87,Digital Extremes,2013-03-25T04:00:00.000Z,2013-03-25,3.42,No age rating,"Action, Shooter, RPG, Massively Multiplayer","iOS, Xbox Series S/X, PlayStation 5, PlayStati..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5711,Mirror Maker,83137,1.99,65436,71,KAGAMI WORKs,2019-01-17T05:00:00.000Z,2019-01-24,1.75,No age rating,"RPG, Simulation, Indie",PC
5712,Kromaia,10378,9.99,65277,75,Kraken Empire,2014-10-23T04:00:00.000Z,2014-10-22,0.00,No age rating,"Action, Shooter, Arcade, Indie","PlayStation 4, PC"
5713,DUCATI - 90th Anniversary,11258,9.99,65188,74,Milestone S.r.l.,2016-06-09T04:00:00.000Z,2016-06-09,2.60,No age rating,"Simulation, Sports, Racing","PlayStation 4, PC, Xbox One"
5714,The Chaos Engine,26117,9.99,65092,68,"Abstraction Games,The Bitmap Brothers",2013-08-29T04:00:00.000Z,1993-03-01,3.67,No age rating,"Action, Shooter","Atari ST, Classic Macintosh, SNES, Commodore /..."


In [72]:
cleaned = joined[joined['rating'] != 0] # Удаляем строки, где рейтинг равен 0, так как они не имеют ценности для анализа зависимости от выручки с игры
cleaned

,name,copiesSold,price,revenue,reviewScore,developers,firstReleaseDate,released,rating,age_rating,genres,platforms
0,Counter-Strike 2,324828766,0.00,9898329661,86,Valve,2012-05-01T00:00:00.000Z,2023-09-27,3.58,Mature,Shooter,"Linux, PC"
1,Dota 2,213939465,0.00,2091450944,81,Valve,2011-08-01T00:00:00.000Z,2013-07-09,3.07,No age rating,"Action, Massively Multiplayer","Linux, macOS, PC"
2,Apex Legends,72503084,0.00,1491767840,67,Respawn,2020-11-04T05:00:00.000Z,2019-02-04,3.64,Teen,"Action, Shooter","PlayStation 4, Nintendo Switch, macOS, PC, Xbo..."
3,Destiny 2,38861886,0.00,901439163,78,Bungie,2019-10-01T04:00:00.000Z,2017-09-06,3.52,Teen,"Action, Shooter","PlayStation 4, Xbox One, PC, Web, Xbox Series ..."
4,Warframe,47103348,0.00,879275446,87,Digital Extremes,2013-03-25T04:00:00.000Z,2013-03-25,3.42,No age rating,"Action, Shooter, RPG, Massively Multiplayer","iOS, Xbox Series S/X, PlayStation 5, PlayStati..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5710,American Conquest,30326,4.99,65545,68,GSC Game World,2011-08-27T04:00:00.000Z,2002-11-15,3.68,No age rating,Strategy,PC
5711,Mirror Maker,83137,1.99,65436,71,KAGAMI WORKs,2019-01-17T05:00:00.000Z,2019-01-24,1.75,No age rating,"RPG, Simulation, Indie",PC
5713,DUCATI - 90th Anniversary,11258,9.99,65188,74,Milestone S.r.l.,2016-06-09T04:00:00.000Z,2016-06-09,2.60,No age rating,"Simulation, Sports, Racing","PlayStation 4, PC, Xbox One"
5714,The Chaos Engine,26117,9.99,65092,68,"Abstraction Games,The Bitmap Brothers",2013-08-29T04:00:00.000Z,1993-03-01,3.67,No age rating,"Action, Shooter","Atari ST, Classic Macintosh, SNES, Commodore /..."
